In [1]:
#export
import torch.nn as nn
from Functions import *

In [2]:
# Load all the data from the CSV file 
PT_DATA_PATH = "../../../Dataset/Botnet_Detection/PT_838_Security Camera"
PT2_DATA_PATH = "../../../Dataset/Botnet_Detection/PT737E_Security Camera"
XC_DATA_PATH = "../../../Dataset/Botnet_Detection/XCS7_1002_WHT_Security_Camera"
#BM_DATA_PATH = "../../../Dataset/Botnet_Detection/Philips_B120N10_Baby_Monitor"
df_pt_1 = pd.read_csv(PT_DATA_PATH+"/benign_traffic.csv")
df_pt_2 = pd.read_csv(PT2_DATA_PATH+"/benign_traffic.csv")
df_xc =  pd.read_csv(XC_DATA_PATH+"/benign_traffic.csv")

In [3]:
df_p1 = df_pt_1.assign(label = 'pt1')
df_pt2 = df_pt_2.assign(label = 'pt2')
df_xc = df_xc.assign(label = 'xc')

In [4]:
df_all = df_p1
df_all = df_all.append(df_pt2, ignore_index = True)

In [5]:
df_all = shuffler(df_all)

In [6]:
#Divide dataframe into x and y
df_x = pd.DataFrame(df_all.iloc[:, 0:115])
df_y = pd.DataFrame(df_all.iloc[:, 115])
df_xc_x = pd.DataFrame(df_xc.iloc[:, 0:115])
df_xc_y = pd.DataFrame(df_xc.iloc[:, 115])

In [8]:
#Normalize the x dataframe
df_x = normalize(df_x)
df_xc = normalize(df_xc_x)

In [9]:
#One-Hot encoding labels and transform into array
df_y = label_encoder(df_y)

[array(['pt1', 'pt2'], dtype=object)]


In [10]:
df_y

array([[0., 1.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [0., 1.]])

In [11]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(df_x, df_y, test_size=0.40)

In [12]:
class Net(torch.nn.Module):
    def __init__(self, in_dim, h_dim, out_dim):
        super(Net, self).__init__()
        self.linear1 = torch.nn.Linear(in_dim, h_dim)
        self.bn1 = nn.BatchNorm1d(h_dim)
        self.linear2 = torch.nn.Linear(h_dim, out_dim)
        
    def forward(self, x):
        h_relu = self.linear1(x).clamp(min=0)
        y_pred = self.linear2(h_relu)
        return y_pred

In [13]:
train_x = torch.FloatTensor(train_x.values.astype(np.float32))
test_x = torch.tensor(test_x.values.astype(np.float32))
train_y = torch.tensor(train_y.astype(np.float32))
test_y = torch.tensor(test_y.astype(np.float32))
xc_x = torch.tensor(df_xc.values.astype(np.float32))

In [14]:
xc_x.shape

torch.Size([46585, 115])

In [15]:
train_x.shape

torch.Size([96400, 115])

In [16]:
test_x.shape

torch.Size([64268, 115])

In [17]:
test_y.shape

torch.Size([64268, 2])

In [18]:
test_x.mean(), test_x.std()

(tensor(0.1402), tensor(0.2044))

In [ ]:
epochs = 600
input_dim = 115
output_dim = 2 #Number of clasees
h_dim = 100
lr_rate = 1e-6

In [ ]:
model = Net(input_dim, h_dim, output_dim)
criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=lr_rate)

In [ ]:
n_test, y_bm = test_y.shape

In [ ]:
def training(epochs, model, train_data, labels, vali_data, target):
    for e in range(int(epochs)):
        y_pred = model(train_data)
        # Compute and print loss
        loss = criterion(y_pred, labels)
        if e % 100 == 99:
            loss_f = float(loss)
            print(e, "Loss:", loss_f)
            total = n_test
            correct = 0.0
            outputs = model(vali_data)
            _b, pred = torch.max(outputs, 1)
            vb, label = torch.max(target, 1)
            correct+= float((pred == label).sum())
            accuracy = float(100*(correct/total))
            print('Accuracy: {:.4f}'.format(accuracy))

        # Zero gradients, perform a backward pass, and update the weights.
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [ ]:
training(epochs, model, train_x, train_y, test_x, test_y) ## Train the initial model on Server

In [19]:
def maxind(V, r):
    res=[]
    for row in V:
        l2 = np.linalg.norm(row)
        row = row.unsqueeze(1)
        row = torch.transpose(row,0,1)       
        row = row.squeeze(0)
        r = r.squeeze(0)
        inner = float(torch.dot(row, r))
        s = float(inner/l2)
        res.append(s)
        res_tensor = torch.FloatTensor(res)
    
    _, i = torch.max(res_tensor, 0)
    return i

In [ ]:
a = torch.FloatTensor([[1,2,3],[4,5,6],[7,8,9],[1,0,2],[2,0,0]])
b = torch.FloatTensor([[1,0,1]])
b.shape

In [ ]:
c = torch.cat([a,b], 0)
print(c)

In [ ]:
maxind(a, b)

In [43]:
## Note: The vector(row) picked from the matrix cannot be re-sized directly.
## y is expected as 2D matrix
def OMP(y, V, sl, ep):
    c_encode = torch.zeros([1, len(V)], dtype=torch.float32)
    s_index = []
    r = y
    k = 1
    while (k <= sl) and (np.linalg.norm(r) >= ep):
        i = int(maxind(V, r))
        s_index.append(i)
        temp = []
        for sk in s_index:
            temp_v = V[sk]
            if len(temp)==0:
                temp=temp_v
                temp = temp.unsqueeze(0)
            else:
                temp_v = V[sk].unsqueeze(0)
                temp = torch.cat((temp, temp_v),0)
               
        
        s_matrix = temp
        s_matrix = s_matrix.type(torch.FloatTensor)
        s_i = torch.pinverse(s_matrix)
        c = torch.mm(y, s_i)
        r = y - torch.mm(c, s_matrix)
        k = k+1
    
    print(s_index)
    v=0
    for sk in s_index:
        c_encode[0][sk] = c[0][v]
        v = v+1
    return c_encode

In [ ]:
OMP(b, a, 20, 0.1)

In [71]:
xc_y = xc_x[90][:].unsqueeze(0)
pt_y = test_x[90][:].unsqueeze(0)

In [74]:
c_xc = OMP(xc_y, train_x[:200][:], 10, 0.001)
c_pt = OMP(pt_y, train_x[:200][:], 10, 0.001)
print(c_xc)
print(c_pt)

[134, 174, 112, 38, 128, 160, 63, 121, 116, 128]
[182, 33, 145, 93, 101, 160, 3, 194, 64, 63]
tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  7.4779e-01,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+

In [75]:
t_xc = xc_y - torch.mm(c_xc, train_x[:200][:])
t_pt = pt_y - torch.mm(c_pt, train_x[:200][:])
print(np.linalg.norm(t_xc))
print(np.linalg.norm(t_pt))

655061.8
0.22126769
